# Optimization Test

### Importing libs and data

In [2235]:
import pandas as pd
import pyomo.environ as pyo
from pathlib import Path
import os

In [2236]:
ROOT = Path(__name__).resolve().parent.parent
FILE = "generic_input_case.xlsx"
FILE_PATH = os.path.join(ROOT, "data", FILE)

In [2237]:
pdata = dict()

with pd.ExcelFile(FILE_PATH) as workbook:
    sheets = workbook.sheet_names
    for sheet in sheets:
        pdata[sheet] = pd.read_excel(workbook, sheet_name=sheet)

c:\Users\Vinicius\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### Data treatment

In [2238]:
pdata["HORIZONTE"] = pdata["HORIZONTE"].drop(["CICLO_LENTO"], axis=1)
pdata["FROTA"] = pdata["FROTA"].dropna(axis=1)
pdata["BD_UP"] = pdata["BD_UP"].dropna(axis=1).drop(["DATA_COLHEITA", "IDADE_FLORESTA", "IMA", "RESERVADO"], axis=1)

In [2239]:
pdata["period_set"] = pdata["HORIZONTE"]["DIA"].unique()

pdata["logistics_set"] = pdata["FROTA"]["TRANSPORTADOR"].unique()

pdata["up_set"] = pdata["BD_UP"]["UP"].unique()
# pdata["farm_set"] = pdata["BD_UP"]["FAZENDA"].unique()
# Não há necessidade da fazenda. Cada UP é única. Posso somar tudo da UP e fazer por fazenda
# Ou posso fazer somente por UP. Nesse caso, farei por UP para tentar ser mais completo

pdata["factory_set"] = pdata["FABRICA"]["FABRICA"].unique()

In [2240]:
logistics_dict = pdata["FROTA"].set_index(["TRANSPORTADOR"]).to_dict()
pdata["min_trucks_param"] = logistics_dict["FROTA_MIN"]
pdata["max_trucks_param"] = logistics_dict["FROTA_MAX"]

grua_dict = pdata["GRUA"].set_index(["TRANSPORTADOR"]).to_dict()
pdata["grua_param"] = grua_dict["QTD_GRUAS"]

up_dict = pdata["BD_UP"].set_index(["UP"]).to_dict()
pdata["density_param"] = up_dict["DB"]
pdata["volume_param"] = up_dict["VOLUME"]
pdata["rsp_param"] = up_dict["RSP"]

factory_dict = pdata["FABRICA"].set_index(["FABRICA", "DIA",]).to_dict()
pdata["min_demand_param"] = factory_dict["DEMANDA_MIN"]
pdata["max_demand_param"] = factory_dict["DEMANDA_MAX"]
pdata["min_rsp_param"] = factory_dict["RSP_MIN"]
pdata["max_rsp_param"] = factory_dict["RSP_MAX"]

route_dict = pdata["ROTA"].set_index(["ORIGEM", "TRANSPORTADOR", "DESTINO"]).to_dict()
pdata["cycle_param"] = route_dict["TEMPO_CICLO"]
route_dict_v2 = pdata["ROTA"].set_index(["TRANSPORTADOR"]).to_dict()
pdata["fitting_box_param"] = route_dict_v2["CAIXA_CARGA"]

In [2241]:
pdata["route_possibilities_set"] = route_dict["TEMPO_CICLO"].keys()

### Create Model

In [2242]:
model = pyo.ConcreteModel()

### Sets

In [2243]:
model.period = pyo.Set(initialize=pdata["period_set"])
model.supplier = pyo.Set(initialize=pdata["logistics_set"])
model.up = pyo.Set(initialize=pdata["up_set"])
model.factory = pyo.Set(initialize=pdata["factory_set"])
model.route_possibilities = pyo.Set(initialize=pdata["route_possibilities_set"])

### Parameters

In [2244]:
model.min_trucks = pyo.Param(model.supplier, initialize=pdata["min_trucks_param"])
model.max_trucks = pyo.Param(model.supplier, initialize=pdata["max_trucks_param"])

model.grua = pyo.Param(model.supplier, initialize=pdata["grua_param"])

model.density = pyo.Param(model.up, initialize=pdata["density_param"])
model.volume = pyo.Param(model.up, initialize=pdata["volume_param"])
model.rsp = pyo.Param(model.up, initialize=pdata["rsp_param"])

model.min_demand = pyo.Param(model.factory, model.period, initialize=pdata["min_demand_param"])
model.max_demand = pyo.Param(model.factory, model.period, initialize=pdata["max_demand_param"])
model.min_rsp = pyo.Param(model.factory, model.period, initialize=pdata["min_rsp_param"])
model.max_rsp = pyo.Param(model.factory, model.period, initialize=pdata["max_rsp_param"])

model.cycle = pyo.Param(model.route_possibilities, initialize=pdata["cycle_param"])
model.fitting_box = pyo.Param(model.supplier, initialize=pdata["fitting_box_param"])

### Variables

In [ ]:
model.amount_of_trucks = pyo.Var(model.up, model.supplier, model.period, domain=pyo.NonNegativeReals)
model.amount_delivery = pyo.Var(model.up, model.supplier, model.factory, model.period, domain=pyo.NonNegativeReals)
model.addressed_supplier = pyo.Var(model.up, model.supplier, model.factory, model.period, domain=pyo.Binary)
model.up_started = pyo.Var(model.up, model.supplier, domain=pyo.Binary)


model.db_min = pyo.Var(model.period, domain=pyo.NonNegativeReals)
model.db_max = pyo.Var(model.period, domain=pyo.NonNegativeReals)

### Constraint

In [2246]:
def raw_material_minimum_delivery(model, f, p):
    eq = sum(
        model.amount_delivery[u, s, f, p]
        for u in model.up
        for s in model.supplier
        if (u, s, f) in model.route_possibilities
     ) >= model.min_demand[f, p]
    
    return eq

model.raw_material_minimum_delivery = pyo.Constraint(
    model.factory, model.period, rule=raw_material_minimum_delivery
)

In [2247]:
def raw_material_maximum_delivery(model, f, p):
    eq = sum(
        model.amount_delivery[u, s, f, p]
        for u in model.up
        for s in model.supplier
        if (u, s, f) in model.route_possibilities
     ) <= model.max_demand[f, p]
    
    return eq

model.raw_material_maximum_delivery = pyo.Constraint(
    model.factory, model.period, rule=raw_material_maximum_delivery
)

In [2248]:
def attend_minimum_rsp(model, f, p):
    total_volume = sum(model.amount_delivery[u, s, f, p] for u in model.up for s in model.supplier)
    rsp_value = sum(
        model.amount_delivery[u, s, f, p] * model.rsp[u]
        for u in model.up
        for s in model.supplier
    )

    eq = rsp_value >= total_volume * model.min_rsp[f, p]
    return eq

model.attend_minimum_rsp = pyo.Constraint(
    model.factory, model.period, rule=attend_minimum_rsp
)

In [2249]:
def attend_maximum_rsp(model, f, p):
    total_volume = sum(model.amount_delivery[u, s, f, p] for u in model.up for s in model.supplier)
    rsp_value = sum(
        model.amount_delivery[u, s, f, p] * model.rsp[u]
        for u in model.up
        for s in model.supplier
    )

    eq = rsp_value <= total_volume * model.max_rsp[f, p]
    return eq

model.attend_maximum_rsp = pyo.Constraint(
    model.factory, model.period, rule=attend_maximum_rsp
)

In [2250]:
def transport_capacity(model, u, s, p):
    volume_up_day = sum(
        model.amount_delivery[u, s, f, p]
        for f in model.factory
        if (u, s, f) in model.route_possibilities
    )

    max_capacity_day = sum(
        model.fitting_box[s] * model.cycle[u, s, f]
        for f in model.factory
        if (u, s, f) in model.route_possibilities
    )

    eq = volume_up_day <= max_capacity_day * model.amount_of_trucks[u, s, p]
    return eq

model.transport_capacity = pyo.Constraint(
    model.up, model.supplier, model.period, rule=transport_capacity
)

In [2251]:
def minimum_amount_of_trucks(model, u, s, p):
    eq = model.amount_of_trucks[u, s, p] >= model.min_trucks[s]
    return eq

model.minimum_amount_of_trucks = pyo.Constraint(
    model.up, model.supplier, model.period, rule=minimum_amount_of_trucks
)

In [2252]:
def maximum_amount_of_trucks(model, u, s, p):
    eq = model.amount_of_trucks[u, s, p] <= model.max_trucks[s]
    return eq

model.maximum_amount_of_trucks = pyo.Constraint(
    model.up, model.supplier, model.period, rule=maximum_amount_of_trucks
)

In [2253]:
# Somente 2 fornecedores operarão do dia devido só termos 2 gruas
def crane_limits(model, s, p):
    eq = sum(
        model.addressed_supplier[u, s, f, p]
        for u in model.up
        for f in model.factory
    ) <= model.grua[s]

    return eq

model.crane_limits = pyo.Constraint(
    model.supplier, model.period, rule=crane_limits
)

In [2254]:
def transport_from_up(model, u, s):
    transported_volume = sum(
        model.amount_delivery[u, s, f, p]
        for f in model.factory
        for p in model.period
        if (u, s, f) in model.route_possibilities
    )

    eq = transported_volume == model.volume[u] * model.up_started[u, s]
    return eq

model.transport_from_up = pyo.Constraint(
    model.up, model.supplier, rule=transport_from_up
)

In [2255]:
def small_up_attendance(model, u):
    volume_up = model.volume[u]

    if volume_up <= 7000:
        eq = sum(
            model.up_started[u, s]
            for s in model.supplier
        ) <= 1
    else:
        eq = pyo.Constraint.Skip
    return eq

model.small_up_attendance = pyo.Constraint(
    model.up, rule=small_up_attendance
)

In [ ]:
def minimum_density(model, u, p):
    M = 20000
    addressing = sum(
        model.addressed_supplier[u, s, f, p]
        for s in model.supplier
        for f in model.factory
        if (u, s, f) in model.route_possibilities
    )

    eq = model.db_min[p] <= model.density[u] +  M * (1 - addressing)
    return eq

model.minimum_density = pyo.Constraint(
    model.up, model.period, rule=minimum_density
)

In [ ]:
def maximum_density(model, u, p):
    M = 20000
    addressing = sum(
        model.addressed_supplier[u, s, f, p]
        for s in model.supplier
        for f in model.factory
        if (u, s, f) in model.route_possibilities
    )

    eq = model.db_max[p] >= model.density[u] +  M * (1 - addressing)
    return eq

model.maximum_density = pyo.Constraint(
    model.up, model.period, rule=maximum_density
)

In [2258]:
def addressing_supplier(model, u, s, f, p):
    M = 200000
    eq = model.amount_delivery[u, s, f, p] <= M * model.addressed_supplier[u, s, f, p]
    return eq

model.addressing_supplier = pyo.Constraint(
    model.route_possibilities, model.period, rule=addressing_supplier
)

### Objetive

In [2265]:
def objetive_function(model):
    # eq = sum(
    #     model.amount_delivery[u, s, f, p]
    #     for u in model.up
    #     for s in model.supplier
    #     for f in model.factory
    #     for p in model.period
    #     if (u, s, f) in model.amount_delivery
    # )
    eq = sum(
        model.db_max[p] - model.db_min[p]
        for p in model.period
    )

    # eq2 =  sum(
    #     model.volume_art[u]
    #     for u in model.up
    # )

    return eq #+ eq2

model.objetive_function = pyo.Objective(
    rule=objetive_function, sense=pyo.minimize
)

### Solve model

In [2266]:
model.write("optimization_test_suzano.lp", io_options={"symbolic_solver_labels":True})

solver = pyo.SolverFactory("appsi_highs")
solver.options['time_limit'] = 600 # segundos
# solver.options['time_limit'] = 300 # segundos
solver.options['mip_rel_gap'] = 0.1
solver.options['mip_abs_gap'] = 1.0
solver.options['presolve'] = 'on'
solver.options['parallel'] = 'on'
solver.solve(model)

load_solution=False and check results.termination_condition and
results.found_feasible_solution() before loading a solution.


{'Problem': [{'Lower bound': 72474.47774935204, 'Upper bound': 310787.8413540537, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'minimize'}], 'Solver': [{'Status': 'aborted', 'Termination condition': 'maxTimeLimit', 'Termination message': 'TerminationCondition.maxTimeLimit'}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [ ]:
# 'Lower bound': 73882.68678836442 Rodando com 5 Min
# 'Lower bound': 72474.47774935204 Rodando com 10 Min

In [2268]:
model.amount_delivery.display()

amount_delivery : Size=2418, Index=up*supplier*factory*period
    Key                              : Lower : Value                 : Upper : Fixed : Stale : Domain
     ('S3AX01', 'Pastori', 'LIM', 1) :     0 :    3207.6000000000004 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 2) :     0 :                   0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 3) :     0 :                   0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 4) :     0 :                   0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 5) :     0 :                   0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 6) :     0 :                3207.6 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 7) :     0 :                   0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 8) :     0 :      

In [2269]:
model.amount_of_trucks.display()

amount_of_trucks : Size=2418, Index=up*supplier*period


    Key                       : Lower : Value : Upper : Fixed : Stale : Domain
     ('S3AX01', 'Pastori', 1) :     0 :  27.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 2) :     0 :  27.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 3) :     0 :  27.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 4) :     0 :  27.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 5) :     0 :  27.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 6) :     0 :  27.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 7) :     0 :  27.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 8) :     0 :  27.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 9) :     0 :  27.0 :  None : False : False : NonNegativeReals
    ('S3AX01', 'Pastori', 10) :     0 :  27.0 :  None : False : False : NonNegativeReals
    ('S3AX01', 'Pastori', 11) :

In [2270]:
model.up_started.display()

up_started : Size=78, Index=up*supplier
    Key                   : Lower : Value : Upper : Fixed : Stale : Domain
    ('S3AX01', 'Pastori') :     0 :   1.0 :     1 : False : False : Binary
    ('S3AX01', 'Rampazo') :     0 :   0.0 :     1 : False : False : Binary
      ('S3AX01', 'Tover') :     0 :   0.0 :     1 : False : False : Binary
    ('S3AX02', 'Pastori') :     0 :   1.0 :     1 : False : False : Binary
    ('S3AX02', 'Rampazo') :     0 :   0.0 :     1 : False : False : Binary
      ('S3AX02', 'Tover') :     0 :   0.0 :     1 : False : False : Binary
    ('S3AX03', 'Pastori') :     0 :   1.0 :     1 : False : False : Binary
    ('S3AX03', 'Rampazo') :     0 :   0.0 :     1 : False : False : Binary
      ('S3AX03', 'Tover') :     0 :   0.0 :     1 : False : False : Binary
    ('S3AX04', 'Pastori') :     0 :   1.0 :     1 : False : False : Binary
    ('S3AX04', 'Rampazo') :     0 :   0.0 :     1 : False : False : Binary
      ('S3AX04', 'Tover') :     0 :   0.0 :     1 : False : 

In [2271]:
model.addressed_supplier.display()

addressed_supplier : Size=2418, Index=up*supplier*factory*period
    Key                              : Lower : Value                 : Upper : Fixed : Stale : Domain
     ('S3AX01', 'Pastori', 'LIM', 1) :     0 :                   1.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 2) :     0 :                   0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 3) :     0 :                   0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 4) :     0 :                   0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 5) :     0 :                   0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 6) :     0 :                   1.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 7) :     0 :                   0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 8) :     0 :                   1.0 :     1 : False : False : Binary
     ('S3AX01', '

In [2272]:
model.db_min.display()

db_min : Size=31, Index=period
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
      1 :     0 : 475.08249720331213 :  None : False : False : NonNegativeReals
      2 :     0 : 484.25382589730725 :  None : False : False : NonNegativeReals
      3 :     0 : 471.88326453558693 :  None : False : False : NonNegativeReals
      4 :     0 : 479.65062513371595 :  None : False : False : NonNegativeReals
      5 :     0 :  467.5533513160681 :  None : False : False : NonNegativeReals
      6 :     0 :  408.3762715753419 :  None : False : False : NonNegativeReals
      7 :     0 :    458.55081807639 :  None : False : False : NonNegativeReals
      8 :     0 : 476.35931165453076 :  None : False : False : NonNegativeReals
      9 :     0 :  485.0017403307593 :  None : False : False : NonNegativeReals
     10 :     0 :  467.5533513160681 :  None : False : False : NonNegativeReals
     11 :     0 : 489.65781411567514 :  None : False : False : NonNegativeReals
     12 :     0 :  

In [2273]:
model.db_max.display()

db_max : Size=31, Index=period
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
      1 :     0 : 10493.203247295103 :  None : False : False : NonNegativeReals
      2 :     0 :  10489.65781411609 :  None : False : False : NonNegativeReals
      3 :     0 : 10494.987422568738 :  None : False : False : NonNegativeReals
      4 :     0 : 10485.300232749723 :  None : False : False : NonNegativeReals
      5 :     0 : 10489.657814115677 :  None : False : False : NonNegativeReals
      6 :     0 : 10493.203247294949 :  None : False : False : NonNegativeReals
      7 :     0 : 10493.203247294949 :  None : False : False : NonNegativeReals
      8 :     0 : 10493.203247294949 :  None : False : False : NonNegativeReals
      9 :     0 : 10493.203247300515 :  None : False : False : NonNegativeReals
     10 :     0 : 10494.987422570126 :  None : False : False : NonNegativeReals
     11 :     0 : 10485.300232749723 :  None : False : False : NonNegativeReals
     12 :     0 : 1